# Notebook based code
You can write code in python in the notebook, ship things from the notebook and get respones, outputs, logs.

## Flyte v1 notebook challenges
1. You cannot run "nested" `dynamic` etc in a notebook
2. Seeing logs, retrieving information and identifying information is hard

## Things to note!
1. Simple init
2. asyncio is native in a notebook
3. nested calls square calls hello
4. Observe phases
5. Observe logs

In [1]:
import flyte
import flyte.git

In [2]:
flyte.init_from_config(flyte.git.config_from_root())

In [3]:
env = flyte.TaskEnvironment(
    name="hello_world",
    image=flyte.Image.from_debian_base(),
)

In [4]:
import asyncio
from typing import List


@env.task
async def say_hello(data: str, lt: List[int]) -> str:
    print(f"Hello, world! - {flyte.ctx().action}")
    return f"Hello {data} {lt}"


@env.task
async def square(i: int = 3) -> int:
    print(flyte.ctx().action)
    return i * i


@env.task
async def say_hello_nested(data: str = "default string") -> str:
    print(f"Hello, nested! - {flyte.ctx().action}", flush=True)
    coros = []
    for i in range(3):
        print(f"Enqueuing, Square({i})", flush=True)
        coros.append(square(i=i))

    vals = await asyncio.gather(*coros)
    print(f"Completed with vals = {vals}", flush=True)
    return await say_hello(data=data, lt=vals)

In [5]:
run = await flyte.run.aio(say_hello_nested, data="jupyter says hello 2!")

15:28:31.238197 WARNING  remote_builder.py:86 -  Image 356633062068.dkr.ecr.us-east-2.amazonaws.com/union/playground:flyte-74d9e119cc0d0868af6ce26d011cbb70     
                         found. Skip building.

15:28:31.248424 WARNING  _deploy.py:160 -  Built Image for environment hello_world, image:                                                                      
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/playground:flyte-74d9e119cc0d0868af6ce26d011cbb70

In [6]:
run.url

'https://playground.canary.unionai.cloud/v2/runs/project/ketan/domain/development/rn9k8bz4gspfkbqglwtx'

In [7]:
run.inputs()

{'data': 'jupyter says hello 2!'}

In [8]:
run.show_logs()

Output()

Scrolled 10 lines of logs.

In [10]:
run.outputs()

('Hello jupyter says hello 2! [0, 1, 4]',)